# Working with sysmlv2 models and the API

## This is a notebook walks through a SysMLv2 create, publish, and interogate cycle when using the API <br> (partially based on the pyMBE kerbal example)

## 1 - Lets make a simple sysmlv2 model and view the model diagram

In [ ]:
package jdehart_api_demo {
    import ScalarValues::*;
    
    part A {
        attribute mass : Real = 30;
    }
    package internal {
        part B {
            part C;
            part D;
    }
    }
    
}

In [ ]:
%viz jdehart_api_demo

## 2 - Now lets publish the model to the API

In [ ]:
%publish jdehart_api_demo

## 3 - Now lets fetch a list of models from the API and create a dropdown box to select a particular model from (Dont look at this stuff... it messy for a slide)

In [ ]:
import requests
import getpass
import math

from __future__ import print_function

import time
import sysml_v2_api_client
from sysml_v2_api_client.rest import ApiException
from pprint import pprint

import json
import networkx as NX
import matplotlib.pyplot as plt

import random
import copy

import pymbe.api as pm
#from pymbe.model_loading import ModelingSession as Session
#from pymbe.interpretation.interpretation import RandomGenerationStrategy as RGS

In [ ]:
import pymbe.query as pmQuery

## 4 - Configure API Server Connection

In [ ]:
sysml_api_base_url = 'http://sysml2.intercax.com:9000' # http://sysml2-sst.intercax.com:9000

Activate APIs - Connect the API classes to the server

In [ ]:
configuration = sysml_v2_api_client.Configuration(
    host = sysml_api_base_url
)

projects_api_instance = None

with sysml_v2_api_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    project_api_instance = sysml_v2_api_client.ProjectApi(api_client)
    
commits_api_instance = None

with sysml_v2_api_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    commits_api_instance = sysml_v2_api_client.CommitApi(api_client)
    
elements_api_instance = None

with sysml_v2_api_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    elements_api_instance = sysml_v2_api_client.ElementApi(api_client)

Pull down commits and elements catalogs - With the API handles, use the pre-built methods to get lists of commits and elements.

In [ ]:
project_api_instance

## 5  Now get a list of my projects from the API server

In [ ]:
q_string = "jdehart"
my_projects = [my_proj for my_proj in project_api_instance.get_projects() if my_proj.name.find(q_string) > -1]
my_projects

## 6 - Create a dropdown box with my projects

In [ ]:
# get the project names into a list
model_names = []
for key, value in enumerate(my_projects):
    model_names.append(my_projects[key].name)
model_names

In [ ]:
# create the dropdown box and populate with my project names
from IPython.display import display
from ipywidgets import Dropdown

def dropdown_eventhandler(change):
    global model_id
    model_id = change.new

option_list = model_names
dropdown = Dropdown(description="Select Project:", options=option_list)
dropdown.observe(dropdown_eventhandler, names='index')
display(dropdown)
### pick the api dem @16:36

In [ ]:
# get the last commit
try:
    # Get commits by project
    commits_response = commits_api_instance.get_commits_by_project(my_projects[model_id].id)
    pprint(commits_response)
except ApiException as e:
    print("Exception when calling CommitApi->get_commits_by_project: %s\n" % e)

In [ ]:
# get a list of elements
elements = []
try:
    # Get commits by project
    elements = elements_api_instance.get_elements_by_project_commit(my_projects[model_id].id, commits_response[0].id)
except ApiException as e:
    print("Exception when calling ElementApi->get_elements_by_project_commit: %s\n" % e)

In [ ]:
len(elements)

In [ ]:
# here is an example of what we are working wiht as JSON file
pprint(elements)

With the elements known we can then get each elements data with an API call

In [ ]:
elements_url = (sysml_api_base_url +
                '/projects/{0}/commits/{1}/elements?page[size]=2000').format(my_projects[model_id].id, commits_response[0].id)

In [ ]:
elements_url

In [ ]:
elements_response = requests.get(
    elements_url
)

In [ ]:
elements_data = elements_response.json()

Now we can show the element data in a tree view

In [ ]:
from IPython.display import JSON
JSON(elements_data)

Lets now  look at the elements in a table view

In [ ]:
import pandas as pd
df = pd.DataFrame(elements_data)
df

Thats a big table... lets narrow it down a bit

In [ ]:
df[["@id","@type","name","owningFeatureMembership","owningRelatedElement","result"]]

Now I would love to change an attribute and push it back up.... but how :)